---

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anon\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\anon\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:


df = pd.read_csv('SPAM.csv')




df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [5]:
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [6]:
df = df.replace(to_replace='None', value=np.nan).dropna()

In [7]:
STOPWORDS  = stopwords.words('english') + ['im']
STOPWORDS.remove('not')
STOPWORDS.remove('no')

lemmatizer = WordNetLemmatizer()

#### Функция очистки и лемматизации текста
def clean_text(text):
  #очистить текст от тегов html и привести к нижнему регистру
  text = BeautifulSoup(text).get_text().lower()

  #очистить текст от ссылок html
  #text = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text)

  #очистить текст от всех символов кроме букв латинского алфавита
  #text = re.sub("[^a-z]", " ", text)

  #разбить текст на слова
  #text_words = text.split()

  #выполнить лематизацию текста и удалить стоп-слова
  #result_words = [lemmatizer.lemmatize(word) for word in text_words if not word in STOPWORDS]
    
  return text

In [8]:
df['Message'] = df['Message'].apply(lambda x: clean_text(x))

C:\Users\anon\AppData\Local\Temp\ipykernel_652\3177496906.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text).get_text().lower()


In [9]:
df['Message']

0       go until jurong point, crazy.. available only ...
1                           ok lar... joking wif u oni...
2       free entry in 2 a wkly comp to win fa cup fina...
3       u dun say so early hor... u c already then say...
4       nah i don't think he goes to usf, he lives aro...
                              ...                        
5567    this is the 2nd time we have tried 2 contact u...
5568                 will ü b going to esplanade fr home?
5569    pity, * was in mood for that. so...any other s...
5570    the guy did some bitching but i acted like i'd...
5571                           rofl. its true to its name
Name: Message, Length: 5572, dtype: object

In [10]:
from sklearn.preprocessing import LabelEncoder
labeles = LabelEncoder()
labels = labeles.fit_transform(df['Category'])

In [11]:
labeles.classes_

array(['ham', 'spam'], dtype=object)

---

In [12]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer, 
    DataCollatorWithPadding,
    AutoModelForSequenceClassification, 
    TrainingArguments, 
    Trainer,
    pipeline
)
import numpy as np

In [13]:
BATCH_SIZE = 32
NUM_PROCS = 32
LR = 0.00005
EPOCHS = 10

In [14]:
id2label = {
   0:'ham', 1:'spam'
}
label2id = {
   0:'ham', 1:'spam'
}

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
#X_train, X_test, y_train, y_test = train_test_split(queries, y, test_size=0.2, random_state=42)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(df['Message'], labels, test_size=0.2, random_state=42)

In [18]:
# from sklearn.utils.class_weight import compute_class_weight
# classes = np.unique(y)
# class_weights = compute_class_weight('balanced', classes=classes, y=y)

In [19]:
from sklearn.utils.class_weight import compute_class_weight
classes = np.unique(labels)
class_weights = compute_class_weight('balanced', classes=classes, y=labels)

In [20]:
import torch
# Преобразуйте веса классов в тензор PyTorch
class_weights = torch.tensor(class_weights, dtype=torch.float)

In [21]:
import pandas as pd
data = pd.DataFrame({'text': X_train, 'label': y_train})
data_test = pd.DataFrame({'text': X_test, 'label': y_test})

In [22]:
data_test["label"].value_counts()

label
0    966
1    149
Name: count, dtype: int64

In [23]:
df_label_1 = data_test[data_test['label'] == 1]

# Выберите 10% строк, где label=0
df_label_0 = data_test[data_test['label'] == 0].sample(frac=0.05)

# Объедините два DataFrame
new_df = pd.concat([df_label_1, df_label_0])

# Перемешайте строки, если необходимо
new_df_data_test = new_df.sample(frac=1).reset_index(drop=True)

In [24]:
df_label_1 = data[data['label'] == 1]

# Выберите 10% строк, где label=0
df_label_0 = data[data['label'] == 0].sample(frac=0.05)

# Объедините два DataFrame
new_df = pd.concat([df_label_1, df_label_0])

# Перемешайте строки, если необходимо
new_df_data = new_df.sample(frac=1).reset_index(drop=True)

In [25]:
from datasets import Dataset
dataset_train = Dataset.from_pandas(data)

In [26]:
dataset_test = Dataset.from_pandas(data_test)

In [27]:
from transformers import AutoTokenizer, MobileBertModel

In [28]:
token = AutoTokenizer.from_pretrained("google/bert_uncased_L-2_H-128_A-2")
model = MobileBertModel.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

You are using a model of type bert to instantiate a model of type mobilebert. This is not supported for all configurations of models and can yield errors.
Some weights of MobileBertModel were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.embedding_transformation.bias', 'embeddings.embedding_transformation.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'enco

In [29]:
class TokenizerWrapper:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    
    def tokenize_function(self, examples):
        return self.tokenizer(
            examples["text"],
            padding="max_length",
            max_length=128,#128
            truncation=True,
        )

In [30]:
tokenizer_wrapper = TokenizerWrapper(token)

In [31]:
tokenized_train = dataset_train.map(
    tokenizer_wrapper.tokenize_function, 
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=2
)

Map (num_proc=2):   0%|          | 0/4457 [00:00<?, ? examples/s]

In [32]:
tokenized_test = dataset_test.map(
    tokenizer_wrapper.tokenize_function, 
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=2
)

Map (num_proc=2):   0%|          | 0/1115 [00:00<?, ? examples/s]

In [34]:
import evaluate
accuracy = evaluate.load('accuracy')

In [35]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [36]:
model = AutoModelForSequenceClassification.from_pretrained(
    "google/bert_uncased_L-2_H-128_A-2",#"google/mobilebert-uncased",  "google/bert_uncased_L-2_H-128_A-2"
    num_labels=2, 
    id2label=id2label, 
    label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

4,386,178 total parameters.
4,386,178 training parameters.


In [38]:
batch_size = 16
training_args = TrainingArguments(
    output_dir="bert_uncased_L-2_H-128_A-2",
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    #fp16=True
)

In [39]:
import torch.nn.functional as F
class_weights=class_weights.to("cuda")
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = F.cross_entropy(logits, labels, weight=class_weights)
        return (loss, outputs) if return_outputs else loss

In [40]:
data_collator = DataCollatorWithPadding(tokenizer=token)

In [41]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=token,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

c:\Users\anon\AppData\Local\Programs\Python\Python310\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [42]:
history = trainer.train()

  0%|          | 0/2790 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

{'eval_loss': 0.09160401672124863, 'eval_accuracy': 0.9883408071748879, 'eval_runtime': 0.6833, 'eval_samples_per_second': 1631.895, 'eval_steps_per_second': 102.451, 'epoch': 1.0}
{'loss': 0.1278, 'grad_norm': 23.255704879760742, 'learning_rate': 0.00016415770609318997, 'epoch': 1.79}


  0%|          | 0/70 [00:00<?, ?it/s]

{'eval_loss': 0.09056468307971954, 'eval_accuracy': 0.9910313901345291, 'eval_runtime': 0.6736, 'eval_samples_per_second': 1655.191, 'eval_steps_per_second': 103.913, 'epoch': 2.0}


...

{'eval_loss': 0.14282935857772827, 'eval_accuracy': 0.9901345291479821, 'eval_runtime': 0.6537, 'eval_samples_per_second': 1705.645, 'eval_steps_per_second': 107.081, 'epoch': 8.0}
{'loss': 0.0002, 'grad_norm': 0.002743934979662299, 'learning_rate': 2.078853046594982e-05, 'epoch': 8.96}


  0%|          | 0/70 [00:00<?, ?it/s]

{'eval_loss': 0.14531202614307404, 'eval_accuracy': 0.9910313901345291, 'eval_runtime': 0.6002, 'eval_samples_per_second': 1857.855, 'eval_steps_per_second': 116.637, 'epoch': 9.0}


  0%|          | 0/70 [00:00<?, ?it/s]

{'eval_loss': 0.14768941700458527, 'eval_accuracy': 0.9901345291479821, 'eval_runtime': 0.6072, 'eval_samples_per_second': 1836.409, 'eval_steps_per_second': 115.29, 'epoch': 10.0}
{'train_runtime': 55.0734, 'train_samples_per_second': 809.283, 'train_steps_per_second': 50.66, 'train_loss': 0.029093978527305803, 'epoch': 10.0}


In [43]:
# # Получаем предсказания
predictions = trainer.predict(tokenized_test)

# # Предсказанные классы
predicted_classes = predictions.predictions.argmax(-1)

  0%|          | 0/70 [00:00<?, ?it/s]

In [44]:
from sklearn import metrics
print("Precision: %f" % metrics.precision_score(data_test["label"], predicted_classes, average='macro'))
print("Recall: %f" % metrics.recall_score(data_test["label"], predicted_classes, average='macro'))
print("F1-Score: %f" % metrics.f1_score(data_test["label"], predicted_classes, average='macro'))

Precision: 0.980633
Recall: 0.980633
F1-Score: 0.980633


In [45]:
from sklearn.metrics import classification_report, confusion_matrix
confusion_matrix(data_test["label"], predicted_classes)

array([[961,   5],
       [  5, 144]], dtype=int64)

In [46]:
from accelerate import Accelerator

accelerator = Accelerator()

In [47]:
accelerator.wait_for_everyone()
accelerator.save_model(model, "./out", max_shard_size="1GB", safe_serialization=True)

In [48]:
!optimum-cli export onnx --model "./bert_uncased_L-2_H-128_A-2/checkpoint-37500"  --task text-classification squad_onnx_128/

"optimum-cli" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
